In [1]:
import pandas as pd
import os
from os import listdir
import numpy as np
import keras
from keras.layers import UpSampling1D
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv1D,Input,concatenate
from keras.layers import MaxPooling1D
from keras.layers import Dense
from keras.layers import LSTM
from keras import backend as K 
K.tensorflow_backend._get_available_gpus()
from keras.layers import Activation
from keras.activations import sigmoid
from keras.optimizers import Adam
import pickle

Using TensorFlow backend.
W0712 11:04:26.546508 140279273273152 deprecation_wrapper.py:119] From /home/cse/faculty/tanima.cse/.conda/envs/fascinet/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0712 11:04:26.552625 140279273273152 deprecation_wrapper.py:119] From /home/cse/faculty/tanima.cse/.conda/envs/fascinet/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0712 11:04:26.554328 140279273273152 deprecation_wrapper.py:119] From /home/cse/faculty/tanima.cse/.conda/envs/fascinet/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0712 11:04:26.748445 140279273273152 deprecation_wrapper.py:119] From /home/cse/faculty/tanima.cse/.conda/envs/fascinet/lib/python3.6/site-p

In [2]:
file='frnds.csv'
o=pd.read_csv(file)
o=o.T
directory='Dataset'
Index_name=[o[i][0] for i in range(42)]
input=[]
output=[]
for name in listdir(directory):
    inp=[[0 for i in range(3072)] for i in range(400)]
    out=[0 for i in range(400)]
    path=directory+'/'+name+'/'+'maskrgb'
    path, dirs, files = next(os.walk(path))
    file_count = len(files)
    filename=directory+'/'+name+'/'+name+'.pkl'
    print(filename,file_count)
    with open(filename, 'rb') as f:
        data = pickle.load(f)
        k=max(list(data.keys()))
        m=file_count
        y=(file_count)//400 +1
        z=0
        print(m,k)
        index_num=Index_name.index(name)
        value=o[index_num][1:m]
        print(data[1][7],value[7])
        for i in range(0,file_count,y):
              if(i+y)>file_count:
                  break
              inp[z]=data[i+1].copy()
              if len(value[i:i+y])!=0:
                  out[z]=max(0,max(value[i:i+y]))
              z+=1
        print(i,z)
    input.append(inp)
    output.append(out)
inp=np.array(input,dtype=object)
out=np.array(output,dtype=object)
X=inp
Y=out

Dataset/Friends54/Friends54.pkl 1776
1776 1776
0.89540905 3
1775 355
Dataset/Friends05/Friends05.pkl 1433
1433 1433
0.67586344 1
1432 358
Dataset/Friends46/Friends46.pkl 1181
1181 1180
0.9309647 1
1179 393
Dataset/Friends16/Friends16.pkl 2253
2253 2251
0.89287287 1
2250 375
Dataset/Friends10/Friends10.pkl 1803
1803 1801
0.9372829 4
1800 360
Dataset/Friends37/Friends37.pkl 1480
1480 1477
0.8390118 1
1476 370
Dataset/Friends17/Friends17.pkl 2691
2691 2689
0.8417628 3
2688 384
Dataset/Friends53/Friends53.pkl 715
715 715
0.14445135 4
714 357
Dataset/Friends09/Friends09.pkl 1432
1432 1429
0.4616588 4
1428 358
Dataset/Friends32/Friends32.pkl 1492
1492 1489
0.9954915 0
1488 373
Dataset/Friends45/Friends45.pkl 997
997 997
0.9835834 2
996 332
Dataset/Friends30/Friends30.pkl 787
787 787
0.99846697 2
786 393
Dataset/Friends01/Friends01.pkl 479
479 479
0.72872937 3
478 239
Dataset/Friends12/Friends12.pkl 1112
1112 1111
0.992759 0
1110 370
Dataset/Friends04/Friends04.pkl 1487
1487 1485
0.92999154 0

In [3]:
def euc_dist_keras(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))
def mynewmodel():
    input_to_model=Input(shape=(400,3072,))
    output1=LSTM(1024,return_sequences=True)(input_to_model)
    p1=Conv1D(2048,1,activation='relu')(input_to_model)
    p2=Conv1D(1024,1,activation='relu')(p1)
    p3=Conv1D(512,1,activation='relu')(p2)
    p4=Conv1D(256,1,activation='relu')(p3)
    p5=MaxPooling1D(1,5)(p4)
    output2=LSTM(1024,return_sequences=True)(p5)
    p6=Conv1D(128,1,activation='relu')(p5)
    p7=Conv1D(64,1,activation='relu')(p6)
    p8=Conv1D(32,1,activation='relu')(p7)
    p9=Conv1D(16,1,activation='relu')(p8)
    p10=MaxPooling1D(1,5)(p9)
    output3=LSTM(1024,return_sequences=True)(p10)
    output=concatenate([output1,output2],axis=-2)
    output=concatenate([output,output3],axis=-2)
    output=MaxPooling1D(1,5)(output)
    p11= LSTM(1024,return_sequences=True)(output)
    p12= UpSampling1D(size=2)(p11)
    p13= UpSampling1D(size=2)(p12)
    output_to_model=Dense(1,activation='sigmoid')(p13)
    model = Model(inputs=input_to_model, outputs=output_to_model)
    model.summary()
    return model

In [4]:
normal_y=[[0 for i in range(400)] for j in range(40)]
for i in range(40):
    m1=min(Y[i])
    m2=max(Y[i])
    p=m2-m1
    print(p,m1,m2)
    for j in range(400):
        normal_y[i][j]=(Y[i][j]-m1)/p
print(normal_y)

4.0 0 4.0
4.0 0 4.0
4 0 4
4.0 0 4.0
4 0 4
3 0 3
4.0 0 4.0
4 0 4
4 0 4
4.0 0 4.0
3.0 0 3.0
3 0 3
4 0 4
4.0 0 4.0
4.0 0 4.0
4 0 4
3.0 0 3.0
4.0 0 4.0
4.0 0 4.0
4.0 0 4.0
4 0 4
3 0 3
4.0 0 4.0
4.0 0 4.0
4 0 4
4 0 4
3 0 3
4.0 0 4.0
4.0 0 4.0
4 0 4
4 0 4
4.0 0 4.0
4 0 4
4.0 0 4.0
4.0 0 4.0
4 0 4
4 0 4
4 0 4
4.0 0 4.0
4 0 4
[[0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [5]:
model=mynewmodel()
opt=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(optimizer=opt,
              loss=euc_dist_keras,)
model.load_weights('400.h5')

W0712 11:07:34.535459 140279273273152 deprecation_wrapper.py:119] From /home/cse/faculty/tanima.cse/.conda/envs/fascinet/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0712 11:07:44.075331 140279273273152 deprecation_wrapper.py:119] From /home/cse/faculty/tanima.cse/.conda/envs/fascinet/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0712 11:08:02.509622 140279273273152 deprecation_wrapper.py:119] From /home/cse/faculty/tanima.cse/.conda/envs/fascinet/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 3072)    0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 400, 2048)    6293504     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 400, 1024)    2098176     conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 400, 512)     524800      conv1d_2[0][0]                   
__________________________________________________________________________________________________
conv1d_4 (

In [6]:
from sklearn.metrics import f1_score
def fscore(x):
        testing=np.expand_dims(X[x],axis=0)
        print(testing.shape)
        ans=model.predict(testing)
        print(ans.shape,Y[x].shape)
        a=np.squeeze(ans[0],axis=1)
        b=Y[x]
        print(a.shape,b.shape)
        aa=[]
        bb=[]
        p=sum(a)/400
        q=sum(b)/400
        for i in range(400):
            if a[i]>=p:
                aa.append(1)
            else:
                aa.append(0)
            if b[i]>=q:
                bb.append(1)
            else:
                bb.append(0)
        #print(aa,bb)
        return f1_score(aa,bb,average='macro')*100,aa,bb

In [19]:
import cv2
import random
def make_summary(name):
    x=Index_name.index(name)
    f,aa,bb=fscore(x)
    path=directory+'/'+name+'/'+'maskrgb'
    path, dirs, files = next(os.walk(path))
    file_count = len(files)
    image_folder = directory+'/'+name+'/'+'rgb'
    video_name = name+'_model.avi'
    video_name2= name+'_original.avi'
    print(name,file_count)
    y=file_count//400+1
    frame = cv2.imread(image_folder+'/1.jpg')
    height, width, layers = frame.shape
    video1 = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'DIVX'), 10, (width,height))
    video2 = cv2.VideoWriter(video_name2,cv2.VideoWriter_fourcc(*'DIVX'), 10, (width,height))
    fake=[]
    original=[]
    for i in range(400):
      if aa[i]==1:
        x1=1+i*y
        if x1>file_count:
          break
        x2=1+(i+1)*y
        if x2>file_count:
          p=x1
        else:
          p1=random.randint(x1,x2)
        fake.append(p1)
      if bb[i]==1:
        x3=1+i*y
        if x3>file_count:
          break
        x4=1+(i+1)*y
        if x4>file_count:
          p=x3
        else:
          p2=random.randint(x3,x4)
        original.append(p2)
    new_fake=[]
    if len(fake)>len(original):
        print('TOOOOOO MUCHHHH')
        x=fake[0]
        for i in range(1,len(fake)):
            if fake[i]-x>y:
                x=fake[i]
                new_fake.append(fake[i])
    else:
        new_fake=fake
    for i in new_fake:
        image=str(i)+'.jpg'
        print(image)
        video1.write(cv2.imread(os.path.join(image_folder, image)))
    cv2.destroyAllWindows()
    video1.release()
    for i in original:
        image=str(i)+'.jpg'
        print(image)
        video2.write(cv2.imread(os.path.join(image_folder, image)))
    cv2.destroyAllWindows()
    video2.release()

In [ ]:
make_summary('Friends53')
make_summary('Friends54')

(1, 400, 3072)
(1, 400, 1) (400,)
(400,) (400,)
Friends53 715
TOOOOOO MUCHHHH
4.jpg
9.jpg
34.jpg
37.jpg
40.jpg
43.jpg
47.jpg
51.jpg
55.jpg
59.jpg
63.jpg
67.jpg
71.jpg
75.jpg
79.jpg
82.jpg
86.jpg
89.jpg
93.jpg
99.jpg
103.jpg
106.jpg
109.jpg
113.jpg
116.jpg
121.jpg
124.jpg
127.jpg
131.jpg
135.jpg
139.jpg
144.jpg
147.jpg
151.jpg
154.jpg
157.jpg
160.jpg
164.jpg
169.jpg
173.jpg
179.jpg
183.jpg
186.jpg
190.jpg
194.jpg
197.jpg
201.jpg
206.jpg
209.jpg
213.jpg
217.jpg
221.jpg
227.jpg
233.jpg
238.jpg
266.jpg
269.jpg
275.jpg
278.jpg
282.jpg
285.jpg
289.jpg
292.jpg
295.jpg
298.jpg
302.jpg
305.jpg
308.jpg
313.jpg
317.jpg
321.jpg
325.jpg
331.jpg
335.jpg
340.jpg
343.jpg
347.jpg
351.jpg
355.jpg
358.jpg
361.jpg
364.jpg
368.jpg
372.jpg
375.jpg
380.jpg
384.jpg
474.jpg
479.jpg
482.jpg
485.jpg
489.jpg
37.jpg
41.jpg
42.jpg
44.jpg
45.jpg
48.jpg
49.jpg
53.jpg
55.jpg
57.jpg
59.jpg
61.jpg
63.jpg
65.jpg
65.jpg
67.jpg
71.jpg
72.jpg
74.jpg
76.jpg
78.jpg
80.jpg
83.jpg
84.jpg
85.jpg
89.jpg
90.jpg
91.jpg
93.jpg
95.jp

In [13]:
len(fake),len(original)

NameError: name 'fake' is not defined

In [17]:
Index_name

['Friends37',
 'Friends36',
 'Friends35',
 'Friends39',
 'Friends38',
 'Friends33',
 'Friends34',
 'Friends31',
 'Friends32',
 'Friends56',
 'Friends40',
 'Friends55',
 'Friends41',
 'Friends42',
 'Friends43',
 'Friends44',
 'Friends45',
 'Friends46',
 'Friends47',
 'Friends48',
 'Friends49',
 'Friends50',
 'Friends51',
 'Friends52',
 'Friends53',
 'Friends54',
 'Friends07',
 'Friends09',
 'Friends10',
 'JohnWick01',
 'JohnWick02',
 'Friends12',
 'Friends13',
 'Friends15',
 'Friends16',
 'Friends17',
 'Friends19',
 'Friends01',
 'Friends02',
 'Friends04',
 'Friends05',
 'Friends30']